<a href="https://colab.research.google.com/github/ChrisKantor/Deep-Learning/blob/main/Homework%203/Deep_Learning_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Deep Learning Assignment Notes:
#Need to submite 3 things, training code, the trained model, and then a bunch of results.
#need to try all of the things listed in q2
#lots of tial and eror with the hyperparameter tuning

#data set is about baby vitals for detecting baby pain
#3 classes: no pain, mild pain, lots of pain
#col 1 is heart rate
#col 2 is respiratory rate
#col 3 is oxygen saturation
#Can merge all the csv files into 1 large one
#randomize and then split into training/validation data

#need to clean the data, invalid data could be in any column

#need 5 HIDDEN layers, so 7 total with the input and output layers
#We can choose any number of neurons per layer

#70% accuracy is a good minimum

#Submit the best model only, but save figures for the other models with hyperparameter tuning
#the model is not enough, need to submit a script which loads the model and then runs the test dataset on it
#the test data has 2 folders same format as training data, need to combine the data into 1 csv

#the testing script can be on a seperate cell in the notebook

#figures are for validation loss
#don't need to combine hyperparameters, just do them without each other, should be 12 figures total for part 2
#1 figure for part 1 (the best model)

# **Loading Dataset**

In [3]:
#import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import os


In [2]:
## Mount Google drive
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/Homework 3 Data/'


mainFile = "baby_data.csv"

#create a new csv file that will hold all of our valid data
with open("/content/drive/MyDrive/Colab Notebooks/Deep Learning/Homework 3 Data/" + mainFile, 'w') as csvfile:
  csvwriter = csv.writer(csvfile)

  # writing the column names
  csvwriter.writerow(['Heart Rate', 'Respiratory Rate', 'O2 Level', 'Label'])

  #use the os library to iterate through all files in the current path
  for (root,dirs,files) in os.walk(data_dir, topdown=True):
    #open each CSV file, we don't want to open any other type of file
    for f in files:
      if f.endswith(".csv"):
        #in each file, go through each row and check if it is valid
        with open(os.path.join(root, f), mode ='r') as csvFile:
          csvData = csv.reader(csvFile)

          #one way to tell if the data is invalid is if the label is NOT [0, 1, 2]. This means there was something wrong with one of the sensors at the time of capture
          #another way is to check if the heart rate, respiratory rate, or o2 level is too low, in this case we check if it is <= 0. if so, there was something wrong with the sensor
          for row in csvData:
            try:
              if row[4] in "012" and int(row[1]) > 0 and int(row[2]) > 0 and int(row[3]) > 0:   #data is valid, so add it to the main csv

                #row[0] just stores the row # in the csv file, so it is not needed in the training dataset
                csvwriter.writerow([row[1], row[2], row[3], row[4]])

            #Catches errors that occur when an element cannot be parsed into an integer. In this case the row is invalid so we can skip it
            except ValueError:
              continue

print("Done writing to main csv file: ", mainFile)

Mounted at /content/drive
Done writing to main csv file:  baby_data.csv


In [4]:
#This function turns the pandas dataframe into a tensorflow dataset to use for training
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  labels = df.pop('Label')
  df = {key: value.values[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds


#This function normalizes NUMERICAL data
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = tf.keras.layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [87]:
#Now we have a csv file that has ALL of the valid data which we will use in training our model
#The next step is to split the data into a training and validation set, using a 90/10 split
dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Deep Learning/Homework 3 Data/" + mainFile, sep=',')
print(dataframe.shape)
print(dataframe.head())


#split the data into a training and validation set using a 90/10 split, and randomize beforehand
trainData, valData = np.split(dataframe.sample(frac=1), [int(0.9*len(dataframe))])

#save the labels to a seperate dataframe and remove them from the data dataframe
trainLabel = trainData.pop('Label')
valLabel = valData.pop('Label')


# #normalize the data using pandas
# for col in trainData.columns:
#   trainData[col] = trainData[col]  / trainData[col].abs().max()

# for col in valData.columns:
#   valData[col] = valData[col]  / valData[col].abs().max()


# #A hyperparameter to tune, batch size
# batch_size = 32
# train_ds = df_to_dataset(train, batch_size=batch_size)
# val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)



# all_inputs = []
# encoded_features = []

# # Numerical features.
# for header in ['Heart Rate', 'Respiratory Rate', 'O2 Level']:
#   numeric_col = tf.keras.Input(shape=(1,), name=header)
#   normalization_layer = get_normalization_layer(header, train_ds)
#   encoded_numeric_col = normalization_layer(numeric_col)
#   all_inputs.append(numeric_col)
#   encoded_features.append(encoded_numeric_col)

# #merge all the features into one vector
# all_features = tf.keras.layers.concatenate(encoded_features)

# print("All Features:\n", all_features)

(16531, 4)
   Heart Rate  Respiratory Rate  O2 Level  Label
0         172                46        98      2
1         172                46        98      2
2         173                46        98      2
3         175                48        98      2
4         172                49        98      2


# Model Creation and Training

In [99]:
#Create the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(3),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(3)
])

#creating the loss function we will use
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='Adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [102]:
#Running the model
print("Running Model\n")
model.fit(trainData, trainLabel, epochs=10, batch_size=32)

print("\n\nEvaluating Model\n")
#evaluation the model on the validation data
model.evaluate(valData, valLabel, verbose=2)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/Deep Learning/Homework 3 Data/baby_pain_model.keras")
print("Saved Model")

Running Model

Epoch 1/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6556 - accuracy: 0.6991
Epoch 2/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6539 - accuracy: 0.6978
Epoch 3/10
465/465 [==============================] - 1s 3ms/step - loss: 0.6542 - accuracy: 0.7021
Epoch 4/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6550 - accuracy: 0.7000
Epoch 5/10
465/465 [==============================] - 2s 3ms/step - loss: 0.6603 - accuracy: 0.7007
Epoch 6/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6526 - accuracy: 0.7011
Epoch 7/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6535 - accuracy: 0.7044
Epoch 8/10
465/465 [==============================] - 3s 6ms/step - loss: 0.6492 - accuracy: 0.7058
Epoch 9/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6492 - accuracy: 0.7038
Epoch 10/10
465/465 [==============================] - 2s 4ms/step - loss: 0.6495 - a